## Bibliotecas

In [3]:
import pandas as pd                             # Dataframes y utilidades
from spacy.lang.es.stop_words import STOP_WORDS # Palabras vacias
import spacy                                    # Procesamiento de lenguaje natural

## Cargar Corpus

In [2]:
corpus = pd.read_excel('apoyo/Rest_Mex_2022.xlsx')
corpus.head()

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel


## Proceso de normalización

In [13]:
nlp = spacy.load("es_core_news_sm")

corpus_tokenizado = []

for contenido in corpus["Opinion"]:
    try:
        doc = nlp(contenido)                        # Analisis de texto
        tokens = [token.text for token in doc]      # Tokenización = separación de palabras
        corpus_tokenizado.append(" ".join(tokens))  # Lista de tokens a cadena de texto
    except:
        print("Error:", contenido)
        print(type(contenido))


Error: nan
<class 'float'>
Error: vacaciones increíble. El personal realmente hizo que estas vacaciones sean inolvidables. Ellos nos atendió manos y pies los 6 días que estuvimos allí. Ellos se acordaban de nosotros (y lo que nos bebimos) y nos recibieron con una sonrisa. Respondieron a preguntas y fueron muy atentos. El entretenimiento nocturno era de primera clase, mejor que he visto en cualquier otro resort. Algunos ejemplos fueron un espectáculo mexicano, espectáculo de Michael Jackson, karaoke, y lo mejor es que era como la historia de la música. Excelente!! Las playas son preciosas y muy limpio. De hecho, todo el complejo estaba impecable. La comida era de primera categoría. Las bebidas eran interminable. Tengo que decir que lo único que me decepcionó fue el 1500 ilimitadas crédito en el complejo. Nos dieron muchas excursiones para elegir. Los precios eran enumerados y nuestra crédito sin cubrir, hasta $1500. Pero luego tienes que pagar el 20% del costo de la excursión como una t